In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation, Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
from imutils import paths
from sklearn.model_selection import train_test_split

In [3]:
dir_path = '/content/drive/MyDrive/dataset'
categories = ['airplane','truck','bird','cat']
data = []

for i in categories:

    paths = os.path.join(dir_path,i)
    print('paths',paths)
    ii=0
    for j in os.listdir(paths):
        img_path = os.path.join(paths,j)
        labels = categories.index(i)

        if (os.path.splitext(img_path)[1]=='.jpg'):
          img = cv2.imread(img_path)
          img = cv2.resize(img,(32,32))
          data.append([img,labels])


random.shuffle(data)
print(len(data))

paths /content/drive/MyDrive/dataset/airplane
paths /content/drive/MyDrive/dataset/truck
paths /content/drive/MyDrive/dataset/bird
paths /content/drive/MyDrive/dataset/cat
80


In [4]:
x = []
y = []

for features,label in data:
    x.append(features)
    y.append(label)

#Converting lists into numpy arrays
x = np.array(x)
y = np.array(y)
x = x/255.0
x = np.array(x).reshape(-1, 32, 32, 3)
print("Shape of train images is:", x.shape)
print("Shape of labels is:", y.shape)
print(y)
print(x.shape[1:])

Shape of train images is: (80, 32, 32, 3)
Shape of labels is: (80,)
[2 2 1 3 2 0 3 3 3 1 3 2 0 0 3 2 0 3 1 0 1 0 0 0 0 1 3 2 3 1 1 2 0 3 1 0 3
 0 0 2 3 1 2 3 2 0 0 0 2 3 0 1 1 3 3 0 0 1 2 1 2 2 1 1 0 1 2 3 3 2 1 1 2 3
 2 3 2 1 1 2]
(32, 32, 3)


In [5]:
model = Sequential()
model.add(Conv2D(64,(3,3), activation='relu', input_shape = (x.shape[1:])))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation='softmax'))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 178,756 (698.27 KB)

 Trainable params: 178,756 (698.27 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x, y, epochs=50,batch_size=10,validation_split=0.2)

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.2482 - loss: 1.4169 - val_accuracy: 0.1875 - val_loss: 1.4058
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4053 - loss: 1.3517 - val_accuracy: 0.0625 - val_loss: 1.4547
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3089 - loss: 1.4147 - val_accuracy: 0.0625 - val_loss: 1.4307
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3435 - loss: 1.3596 - val_accuracy: 0.0625 - val_loss: 1.4641
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.3551 - loss: 1.3226 - val_accuracy: 0.0625 - val_loss: 1.4786
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3674 - loss: 1.3601 - val_accuracy: 0.0625 - val_loss: 1.4607
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3544 - loss: 1.3233 - val_accuracy: 0.0625 - val_loss: 1.4725
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.2333 - loss: 1.3880 - val_accuracy: 0.0625 - val_loss: 1.4464


In [7]:

import numpy as np


def image(path):
    print('path',path)
    img = cv2.imread(path)

    new_arr = cv2.resize(img, (32, 32))
    new_arr = np.array(new_arr)
    new_arr = new_arr/255.0
    new_arr = new_arr.reshape(-1, 32, 32, 3)

    return new_arr



In [8]:
dir_path = '/content/drive/MyDrive/dataset'
categories = ['test_airplane','test_truck','test_bird','test_cat']
data = []

correct_cnt=0
for i in categories:

    paths = os.path.join(dir_path,i)
    ii=0
    for j in os.listdir(paths):
        img_path = os.path.join(paths,j)
      #  print(img_path)
        labels = categories.index(i)
      #  print(img_path,labels)
        prediction = model.predict(image(img_path))
        print('labels' ,labels)
        print('classification results' ,prediction.argmax())
        if (labels==prediction.argmax()):
            correct_cnt=correct_cnt+1
print('### correnct cnt out of 20==>',correct_cnt)

path /content/drive/MyDrive/dataset/test_airplane/0102.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
labels 0
classification results 3
path /content/drive/MyDrive/dataset/test_airplane/0101.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
labels 0
classification results 3
path /content/drive/MyDrive/dataset/test_airplane/0103.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
labels 0
classification results 2
path /content/drive/MyDrive/dataset/test_airplane/0100.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
labels 0
classification results 1
path /content/drive/MyDrive/dataset/test_airplane/0104.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
labels 0
classification results 0
path /content/drive/MyDrive/dataset/test_truck/0101.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
labels 1
classification results 1
path /content/drive/MyDrive/dataset/test_truck/0102.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
labels 1
classification results 1
path /content/drive/MyDrive/dataset/test_truck/0103.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37m